<a href="https://colab.research.google.com/github/Chubbyman2/SSD_MobileNet_Hand_Tracker/blob/main/training_ssd_mobilenet_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Original Code From:

https://medium.com/@matus.tanon/custom-object-detection-using-tensorflow-in-google-colab-e4d6e1a17f18

This will provide you with the frozen_inference_graph.pb needed for the prediction program.

In [ ]:
%tensorflow_version 1.x  # Only runs on ver 1.x of Tensorflow
!pip show tensorflow

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.x  # Only runs on ver 1.x of Tensorflow`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.
Name: tensorflow
Version: 1.15.2
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /tensorflow-1.15.2/python3.6
Requires: astor, protobuf, google-pasta, six, tensorflow-estimator, opt-einsum, tensorboard, keras-preprocessing, numpy, gast, grpcio, termcolor, wheel, absl-py, wrapt, keras-applications
Required-by: stable-baselines, magenta, fancyimpute


In [ ]:
!pip install tf_slim

     |████████████████████████████████| 358kB 6.0MB/s 


In [ ]:
%cd /root/
!git clone https://github.com/tensorflow/models.git # Import required models from Github

/root
Cloning into 'models'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 49668 (delta 18), reused 52 (delta 12), pack-reused 49610
Receiving objects: 100% (49668/49668), 558.73 MiB | 41.58 MiB/s, done.
Resolving deltas: 100% (34251/34251), done.


In [ ]:
# Install Tensorboard
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

--2021-01-03 01:19:54--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.204.23.149, 52.206.211.104, 34.195.187.253, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.204.23.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  52.8MB/s    in 0.2s    

2021-01-03 01:19:54 (52.8 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [ ]:
# The logs that are created while training 
LOG_DIR = "/root/models/trained"
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
get_ipython().system_raw('./ngrok http 6006 &')

In [ ]:
# Quick test of the Object Detection API
%cd /root/models/research/
!protoc object_detection/protos/*.proto --python_out=.
import os
os.environ['PYTHONPATH'] += ':/root/models/research/:/root/models/research/slim/:/root/models/research/object_detection/utils/:/root/models/research/object_detection'
!python object_detection/builders/model_builder_test.py

/root/models/research


In [ ]:
# Download the pre-trained ssd_mobilenet_v1_coco model from Tensorflow model zoo
%cd ~/models

import os
import shutil
import glob
import urllib
import tarfile
from requests import get

MODEL = 'ssd_mobilenet_v1_coco_2017_11_17'
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = 'pretrained_model'

if not (os.path.exists(MODEL_FILE)):
  with open(MODEL_FILE, "wb") as file:
    response = get(DOWNLOAD_BASE + MODEL_FILE)
    file.write(response.content)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
  shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)

/root/models


In [ ]:
# Move Config file to directory "models"
try:
  shutil.move("/root/models/research/object_detection/samples/configs/ssd_mobilenet_v1_coco.config", "/root/models")
except:
  pass

In [ ]:
%cd /root/models/research/
!protoc object_detection/protos/*.proto --python_out=.
import os
os.environ['PYTHONPATH'] += ':/root/models/research/:/root/models/research/slim/:/root/models/research/object_detection/utils/:/root/models/research/object_detection'

# Edit Pipeline 
import tensorflow as tf
from google.protobuf import text_format
from object_detection.protos import pipeline_pb2

pipeline = pipeline_pb2.TrainEvalPipelineConfig()                                                                                                                                                                                                          
config_path = '/root/models/ssd_mobilenet_v1_coco.config'
with tf.gfile.GFile(config_path, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline)

pipeline.train_config.fine_tune_checkpoint = '/root/models/pretrained_model/model.ckpt'
pipeline.train_config.num_steps = 10000 # Feel free to train for more steps
pipeline.model.ssd.num_classes = 1 # Just hands, nothing else
pipeline.eval_config.num_examples = 5

/root/models/research


In [ ]:
import zipfile
from google.colab import drive
drive.mount('/content/drive/')

!unzip "/content/drive/My Drive/egohands_images.zip"

In [ ]:
# Change the first four lines to where the images and label maps are
pipeline.train_input_reader.tf_record_input_reader.input_path[0] = 'egohands_images/train/train.record'
pipeline.train_input_reader.label_map_path = '/content/drive/My Drive/hand_label_map.pbtxt'
pipeline.eval_input_reader[0].tf_record_input_reader.input_path[0] = 'egohands_images/test/test.record'
pipeline.eval_input_reader[0].label_map_path = '/content/drive/My Drive/hand_label_map.pbtxt'

config_text = text_format.MessageToString(pipeline)                                                                                                                                                                                                        
with tf.gfile.Open(config_path, "wb") as f:                                                                                                                                                                                                                       
    f.write(config_text)

In [ ]:
# Compile protobuf and change Python path
%cd /root/models/research/
!protoc object_detection/protos/*.proto --python_out=.
import os
os.environ['PYTHONPATH'] += ':/root/models/research/:/root/models/research/slim/:/root/models/research/object_detection/utils/:/root/models/research/object_detection'

# Begin training
!python /root/models/research/object_detection/legacy/train.py \
    --logtostderr \
    --train_dir=/root/models/trained \
    --pipeline_config_path=/root/models/ssd_mobilenet_v1_coco.config

In [ ]:
# Export trained model 
%cd /root/models/research
!protoc object_detection/protos/*.proto --python_out=.
import os
os.environ['PYTHONPATH'] += ':/root/models/research/:/root/models/research/slim/:/root/models/research/object_detection/utils/:/root/models/research/object_detection'
!python /root/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path=/root/models/ssd_mobilenet_v1_coco.config \
    --output_directory=/root/models/fine_tuned_model \
    --trained_checkpoint_prefix=/root/models/trained/model.ckpt-10000 # Change this depending on num of training steps

In [17]:
# The link to tensorboard works after training finishes
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://31e73d722568.ngrok.io
